### Exercise 1

* create document Task with name, description, priority(low, normal, urgent), Size(S, M, L), Status(Backlog, TODO, InProgress, OnHold, Review, Done)
* add to User Document recently task added by user  
* add one user & task

In [12]:
from enum import IntEnum
from typing import Optional
from beanie import Document
from pydantic import BaseModel
import os

from beanie import init_beanie
from motor.motor_asyncio import AsyncIOMotorClient


class PriorityType(IntEnum):
    low = 1
    normal = 2
    urgent = 3


class SizeType(IntEnum):
    S = 1
    M = 2
    L = 3


class StatusType(IntEnum):
    BACKLOG = 1
    TODO = 2
    InProgress = 3
    OnHold = 4
    Review = 5
    Done = 6


class Task(Document):
    name: str
    description: Optional[str] = None
    priority: Optional[PriorityType] = None
    size: Optional[SizeType] = None
    status: StatusType = StatusType.BACKLOG

class Active(BaseModel):
    active: bool = True


class Address(BaseModel):
    country: str
    city: str
    street: str
    building_number: str
    zip_code: str


class User(Document, Active):
    name: str
    surname: str
    email: str
    address: Optional[Address] = None
    recently_tasks: Optional[Task] = None


client = AsyncIOMotorClient(os.getenv("MONGODB_URI"))

await init_beanie(
    database=client.workshop,
    document_models=[User, Task],
    multiprocessing_mode=True,
)
client.drop_database(name_or_database=client.workshop)

upper_task = Task(name='Call to My Momma', priority=PriorityType.urgent, size=SizeType.L, status=StatusType.InProgress)
hot_adam = User(name="Adam", surname="Brzyzek", email="hotadam@gmail.com", recently_tasks=upper_task)
await User.insert(hot_adam)
await Task.insert(upper_task)


Task(id=ObjectId('66c9e00c9721746de2ec9ecd'), revision_id=None, name='Call to My Momma', description=None, priority=<PriorityType.urgent: 3>, size=<SizeType.L: 3>, status=<StatusType.InProgress: 3>)

In [14]:
await User.find().to_list()

[User(active=True, id=ObjectId('66c9e00c9721746de2ec9ecc'), revision_id=None, name='Adam', surname='Brzyzek', email='hotadam@gmail.com', address=None, recently_tasks=Task(id=None, revision_id=None, name='Call to My Momma', description=None, priority=<PriorityType.urgent: 3>, size=<SizeType.L: 3>, status=<StatusType.InProgress: 3>))]

In [15]:
await Task.find().to_list()


[Task(id=ObjectId('66c9e00c9721746de2ec9ecd'), revision_id=None, name='Call to My Momma', description=None, priority=<PriorityType.urgent: 3>, size=<SizeType.L: 3>, status=<StatusType.InProgress: 3>)]

Show relation
link task with user

TODO explain Link

In [31]:
from beanie import Link


class User(Document, Active):
    name: str
    surname: str
    email: str
    address: Optional[Address] = None
    recently_tasks: Optional[list[Task]] = None


class Task(Document):
    name: str
    description: Optional[str] = None
    priority: Optional[PriorityType] = None
    size: Optional[SizeType] = None
    status: StatusType = StatusType.BACKLOG
    user: Optional[Link[User]] = None


In [33]:
await init_beanie(
    database=client.workshop,
    document_models=[User, Task],
    multiprocessing_mode=True,
)
client.drop_database(name_or_database=client.workshop)

hot_adam = User(
    name="Adam", surname="Brzyzek", email="hotadam@gmail.com"
)
await User.insert(hot_adam)
upper_task = Task(
    name="Call to My Momma",
    priority=PriorityType.urgent,
    size=SizeType.L,
    status=StatusType.InProgress,
)
hot_adam.recently_tasks = [upper_task]
await hot_adam.save()
await Task.insert(upper_task)
upper_task.user = hot_adam
await upper_task.save()



Task(id=ObjectId('66c9f35d9721746de2ec9edf'), revision_id=None, name='Call to My Momma', description=None, priority=<PriorityType.urgent: 3>, size=<SizeType.L: 3>, status=<StatusType.InProgress: 3>, user=User(active=True, id=ObjectId('66c9f35d9721746de2ec9ede'), revision_id=None, name='Adam', surname='Brzyzek', email='hotadam@gmail.com', address=None, recently_tasks=[Task(id=None, revision_id=None, name='Call to My Momma', description=None, priority=<PriorityType.urgent: 3>, size=<SizeType.L: 3>, status=<StatusType.InProgress: 3>, user=None)]))

### Exercise 2

* create document TaskLogStatus for log task status, 
  needs to have priority, size, status, date, link to user and task
* add extend tables with technical tables like active, create_data & update_data
  
